## PARCEIROS 360

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 24/10/2022
- <b>Descrição</b>:

In [52]:
!python --version

Python 3.8.16


### BIBLIOTECAS:

In [53]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [54]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [55]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### FUNÇÕES

In [56]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

In [57]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET

def makedf(base_treino_scaled, dados, kpi_nome, base_treino):
    
    kmeans = KMeans(n_clusters = 3, max_iter = 300, init = 'k-means++', n_init = 10, random_state = 0)
    clusters = kmeans.fit_predict(base_treino_scaled)

    model = RandomForestClassifier(random_state=0)
    model.fit(base_treino_scaled,clusters)
    prediction, bias, contributions = ti.predict(model, base_treino_scaled)

    contributions = np.round(contributions,5)*1000

    contributions_c1 = []
    contributions_c2 = []
    contributions_c3 = []
    contributions_filtered = []

    for i in range(len(contributions)):
        contributions_c1.append([contributions[i][0][0], contributions[i][1][0], contributions[i][2][0], contributions[i][3][0], contributions[i][4][0], contributions[i][5][0], contributions[i][6][0], contributions[i][7][0]])
        contributions_c2.append([contributions[i][0][1], contributions[i][1][1], contributions[i][2][1], contributions[i][3][1], contributions[i][4][1], contributions[i][5][1], contributions[i][6][1], contributions[i][7][1]])
        contributions_c3.append([contributions[i][0][2], contributions[i][1][2], contributions[i][2][2], contributions[i][3][2], contributions[i][4][2], contributions[i][5][2], contributions[i][6][2], contributions[i][7][2]])
        
    for i in range(len(clusters)):
        if clusters[i] == 0:
            contributions_filtered.append(contributions_c1[i])
        if clusters[i] == 1:
            contributions_filtered.append(contributions_c2[i])
        if clusters[i] == 2:
            contributions_filtered.append(contributions_c3[i])

    scaler = MinMaxScaler(feature_range=(0,100))
    contributions_filtered_scaled = scaler.fit_transform(contributions_filtered)

    contributions_filtered = pd.DataFrame(contributions_filtered_scaled, columns=base_treino_scaled.columns + '_CONTRIBUTION')

    base_treino['KPI_original'] = dados[kpi_nome].values
    base_treino['resultado_modelo'] = clusters
    base_treino['resultado_modelo'] = base_treino['resultado_modelo'].map({0:'BOM', 1:'MEDIO', 2:'RUIM'})

    base_treino.reset_index(inplace=True, drop=False)

    results = pd.concat([base_treino, contributions_filtered], axis=1)
    return results

### DADOS:

In [58]:
PATH = "intermediate/"

***
#### PAGAMENTO:

In [65]:
FILE = "etl_previsao_pagamentos.xlsx"
etl_previsao_pagamentos = pd.read_excel(PATH + FILE)
etl_previsao_pagamentos['RAIZ_CNPJ'] = etl_previsao_pagamentos['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
etl_previsao_pagamentos = etl_previsao_pagamentos.drop(['Unnamed: 0'], axis=1)
# etl_previsao_pagamentos = etl_previsao_pagamentos.drop(['Unnamed: 0', 'FORNECEDOR', 'AREA_ATUACAO'], axis=1)

etl_previsao_pagamentos = etl_previsao_pagamentos.drop_duplicates().reset_index(drop=True)

etl_previsao_pagamentos

,FORNECEDOR,MONT_EM_MI,TP_PAGTO_BANCO,TP_PAGTO_FACTORING,TP_PAGTO_FACTORING_MONT_EM_MI,BLOQUEIO_NAO,BLOQUEIO_NAO_MONT_EM_MI,BLOQUEIO_SIM,BLOQUEIO_SIM_MONT_EM_MI,RAIZ_CNPJ,NOME_FORNECEDOR,AREA_ATUACAO,COMPETENCIA,KPI_PGTO
0,10000705,-214698.81,33,0,0.00,33,-214698.81,0,0.00,02641663,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-01-01,BOM
1,10000705,-214698.81,33,0,0.00,33,-214698.81,0,0.00,02641663,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-02-01,BOM
2,10000705,-214698.81,33,0,0.00,33,-214698.81,0,0.00,02641663,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-03-01,BOM
3,10000705,-214698.81,33,0,0.00,33,-214698.81,0,0.00,02641663,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-04-01,BOM
4,10000705,-214698.81,33,0,0.00,33,-214698.81,0,0.00,02641663,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-05-01,BOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8388,14066856,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,29090742,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-03-01,RUIM
8389,14066856,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,29090742,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-04-01,RUIM
8390,14066856,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,29090742,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-05-01,RUIM
8391,14066856,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,29090742,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-06-01,RUIM


In [66]:
etl_previsao_pagamentos[etl_previsao_pagamentos['RAIZ_CNPJ'] == '33067745'] #33067745

,FORNECEDOR,MONT_EM_MI,TP_PAGTO_BANCO,TP_PAGTO_FACTORING,TP_PAGTO_FACTORING_MONT_EM_MI,BLOQUEIO_NAO,BLOQUEIO_NAO_MONT_EM_MI,BLOQUEIO_SIM,BLOQUEIO_SIM_MONT_EM_MI,RAIZ_CNPJ,NOME_FORNECEDOR,AREA_ATUACAO,COMPETENCIA,KPI_PGTO
314,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-01-01,RUIM
315,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-02-01,RUIM
316,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-03-01,RUIM
317,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-04-01,RUIM
318,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-05-01,RUIM
319,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-06-01,RUIM
320,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-07-01,RUIM
321,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-08-01,RUIM
322,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-09-01,RUIM
323,14000049,-1.967052e+08,5339,4553,-1.031838e+08,5339,-9.352137e+07,4553,-1.031838e+08,33067745,ERICSSON TELECOMUNICAÃÃES S/A,TELECOMUNICAÃÃES,2021-10-01,MEDIO


In [48]:
dados = etl_previsao_pagamentos.copy()
dados.set_index(['RAIZ_CNPJ', 'FORNECEDOR', 'NOME_FORNECEDOR', 'AREA_ATUACAO', 'COMPETENCIA'], inplace=True)
dados

MONT_EM_MI  TP_PAGTO_BANCO  TP_PAGTO_FACTORING  TP_PAGTO_FACTORING_MONT_EM_MI  BLOQUEIO_NAO  BLOQUEIO_NAO_MONT_EM_MI  BLOQUEIO_SIM  BLOQUEIO_SIM_MONT_EM_MI KPI_PGTO
RAIZ_CNPJ FORNECEDOR NOME_FORNECEDOR                      AREA_ATUACAO COMPETENCIA                                                                                                                                                                      
2641663   10000705   FUNDAÃÃO CPQD - CENTRO DE PESQUISA CONSULTORIA  2021-01-01   -214698.81              33                   0                           0.00            33               -214698.81             0                     0.00      BOM
                                                                       2021-02-01   -214698.81              33                   0                           0.00            33               -214698.81             0                     0.00      BOM
                                                                       2021-03-01   -214698.81              33                   0                           0.00            33               -214698.81             0                     0.00      BOM
                                                                       2021-04-01   -214698.81              33                   0                           0.00            33               -214698.81             0                     0.00      BOM
                                                                       2021-05-01   -214698.81              33                   0                           0.00            33               -214698.81             0                     0.00      BOM
...                                                                                        ...             ...                 ...                            ...           ...                      ...           ...                      ...      ...
29090742  14066856   M2 SERVICOS DE MENSAGERIA            TRANSPORTES  2022-03-01   -919685.02               0                  85                     -919685.02             0                     0.00            85               -919685.02     RUIM
                                                                       2022-04-01   -919685.02               0                  85                     -919685.02             0                     0.00            85               -919685.02     RUIM
                                                                       2022-05-01   -919685.02               0                  85                     -919685.02             0                     0.00            85               -919685.02     RUIM
                                                                       2022-06-01   -919685.02               0                  85                     -919685.02             0                     0.00            85               -919685.02     RUIM
                                                                       2022-07-01   -919685.02               0                  85                     -919685.02             0                     0.00            85               -919685.02     RUIM

[8393 rows x 9 columns]

In [41]:
dados.columns

Index(['MONT_EM_MI', 'TP_PAGTO_BANCO', 'TP_PAGTO_FACTORING', 'TP_PAGTO_FACTORING_MONT_EM_MI', 'BLOQUEIO_NAO', 'BLOQUEIO_NAO_MONT_EM_MI', 'BLOQUEIO_SIM', 'BLOQUEIO_SIM_MONT_EM_MI', 'KPI_PGTO'], dtype='object')

In [42]:
base_treino_kpi = dados[['MONT_EM_MI', 'TP_PAGTO_BANCO', 'TP_PAGTO_FACTORING', 'TP_PAGTO_FACTORING_MONT_EM_MI', 
                                     'BLOQUEIO_NAO', 'BLOQUEIO_NAO_MONT_EM_MI', 'BLOQUEIO_SIM', 'BLOQUEIO_SIM_MONT_EM_MI']]

In [43]:
base_treino_kpi_scaled = scalers(base_treino_kpi)

base_treino_kpi_scaled

MONT_EM_MI  TP_PAGTO_BANCO  TP_PAGTO_FACTORING  TP_PAGTO_FACTORING_MONT_EM_MI  BLOQUEIO_NAO  BLOQUEIO_NAO_MONT_EM_MI  BLOQUEIO_SIM  BLOQUEIO_SIM_MONT_EM_MI
RAIZ_CNPJ FORNECEDOR NOME_FORNECEDOR                      AREA_ATUACAO COMPETENCIA                                                                                                                                                             
2641663   10000705   FUNDAÃÃO CPQD - CENTRO DE PESQUISA CONSULTORIA  2021-01-01     0.999897        0.000316            0.000000                       0.999999      0.000316                 0.999897      0.000000                 0.999997
                                                                       2021-02-01     0.999897        0.000316            0.000000                       0.999999      0.000316                 0.999897      0.000000                 0.999997
                                                                       2021-03-01     0.999897        0.000316            0.000000                       0.999999      0.000316                 0.999897      0.000000                 0.999997
                                                                       2021-04-01     0.999897        0.000316            0.000000                       0.999999      0.000316                 0.999897      0.000000                 0.999997
                                                                       2021-05-01     0.999897        0.000316            0.000000                       0.999999      0.000316                 0.999897      0.000000                 0.999997
...                                                                                        ...             ...                 ...                            ...           ...                      ...           ...                      ...
29090742  14066856   M2 SERVICOS DE MENSAGERIA            TRANSPORTES  2022-03-01     0.999668        0.000000            0.001881                       0.999467      0.000000                 0.999967      0.001881                 0.999466
                                                                       2022-04-01     0.999668        0.000000            0.001881                       0.999467      0.000000                 0.999967      0.001881                 0.999466
                                                                       2022-05-01     0.999668        0.000000            0.001881                       0.999467      0.000000                 0.999967      0.001881                 0.999466
                                                                       2022-06-01     0.999668        0.000000            0.001881                       0.999467      0.000000                 0.999967      0.001881                 0.999466
                                                                       2022-07-01     0.999668        0.000000            0.001881                       0.999467      0.000000                 0.999967      0.001881                 0.999466

[8393 rows x 8 columns]

***
### CLUSTERING COM K-Means:

In [44]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI, scaled
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET
#base_treino_kpi = BASE DE TREINO, SEM A COLUNA DO KPI


results_kpi = makedf(base_treino_kpi_scaled, dados, 'KPI_PGTO', base_treino_kpi)
results_kpi

,RAIZ_CNPJ,FORNECEDOR,NOME_FORNECEDOR,AREA_ATUACAO,COMPETENCIA,MONT_EM_MI,TP_PAGTO_BANCO,TP_PAGTO_FACTORING,TP_PAGTO_FACTORING_MONT_EM_MI,BLOQUEIO_NAO,BLOQUEIO_NAO_MONT_EM_MI,BLOQUEIO_SIM,BLOQUEIO_SIM_MONT_EM_MI,KPI_original,resultado_modelo,MONT_EM_MI_CONTRIBUTION,TP_PAGTO_BANCO_CONTRIBUTION,TP_PAGTO_FACTORING_CONTRIBUTION,TP_PAGTO_FACTORING_MONT_EM_MI_CONTRIBUTION,BLOQUEIO_NAO_CONTRIBUTION,BLOQUEIO_NAO_MONT_EM_MI_CONTRIBUTION,BLOQUEIO_SIM_CONTRIBUTION,BLOQUEIO_SIM_MONT_EM_MI_CONTRIBUTION
0,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-01-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
1,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-02-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
2,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-03-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
3,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-04-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
4,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-05-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8388,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-03-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746
8389,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-04-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746
8390,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-05-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746
8391,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-06-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746


In [45]:
results_kpi = results_kpi.drop_duplicates().reset_index(drop=True)
results_kpi

,RAIZ_CNPJ,FORNECEDOR,NOME_FORNECEDOR,AREA_ATUACAO,COMPETENCIA,MONT_EM_MI,TP_PAGTO_BANCO,TP_PAGTO_FACTORING,TP_PAGTO_FACTORING_MONT_EM_MI,BLOQUEIO_NAO,BLOQUEIO_NAO_MONT_EM_MI,BLOQUEIO_SIM,BLOQUEIO_SIM_MONT_EM_MI,KPI_original,resultado_modelo,MONT_EM_MI_CONTRIBUTION,TP_PAGTO_BANCO_CONTRIBUTION,TP_PAGTO_FACTORING_CONTRIBUTION,TP_PAGTO_FACTORING_MONT_EM_MI_CONTRIBUTION,BLOQUEIO_NAO_CONTRIBUTION,BLOQUEIO_NAO_MONT_EM_MI_CONTRIBUTION,BLOQUEIO_SIM_CONTRIBUTION,BLOQUEIO_SIM_MONT_EM_MI_CONTRIBUTION
0,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-01-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
1,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-02-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
2,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-03-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
3,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-04-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
4,2641663,10000705,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,CONSULTORIA,2021-05-01,-214698.81,33,0,0.00,33,-214698.81,0,0.00,BOM,BOM,36.110254,5.293724,0.00000,39.381890,3.897120,39.699986,5.293182,41.360098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8388,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-03-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746
8389,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-04-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746
8390,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-05-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746
8391,29090742,14066856,M2 SERVICOS DE MENSAGERIA,TRANSPORTES,2022-06-01,-919685.02,0,85,-919685.02,0,0.00,85,-919685.02,RUIM,BOM,36.087956,5.279645,0.01855,39.386409,3.813251,39.679600,5.413481,41.418746


In [46]:
results_kpi.to_excel(r'outputs/previsao_pagamentos.xlsx')